In [1]:
import os
while not os.path.exists('.gitignore'):
	os.chdir(os.path.abspath(os.path.join('../')))
from common_imports import *

In [2]:
response = httpx.get('https://www.bcb.gov.br/api/paginasite/sitebcb/controleinflacao/historicometas').json()['conteudo']

In [3]:
soup = BeautifulSoup(f'<div>{response}</div>', 'html.parser')

In [4]:
table = soup.find('table')

In [5]:
headers = []
for th in table.find_all('th'):
	headers.append(th.text.strip())

rows = []
for tr in table.find_all('tr')[2:]:
	cells = []
	for td in tr.find_all('td'):
		cell_text = [
			cell.replace('\u200b', '').replace('\n', '').strip()
			for cell in td.decode_contents().split('<br/>')
		]
		cell_text = [cell for cell in cell_text if cell]
		if len(cell_text) == 0:
			cells.append(np.nan)
		else:
			cells.append(cell_text[-1])
	rows.append(cells)

In [6]:
num_cols = len(headers) if headers else max(len(row) for row in rows)

for i, row in enumerate(rows):
    if len(row) < num_cols:
        while len(row) < num_cols:
            row.insert(1, np.nan)

In [7]:
pd.DataFrame(rows, columns=headers if headers else None).to_csv('_data/raw/taxas-inflacao/bcb_metas_inflacao.csv', index=False, sep=';')